In [ ]:
# dataset
!git clone https://github.com/MichaelCai311/keras-multi-label

Cloning into 'keras-multi-label'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 217 (delta 6), reused 201 (delta 0), pack-reused 0
Receiving objects: 100% (217/217), 31.33 MiB | 35.14 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization,Conv2D ,MaxPooling2D ,Activation ,Flatten ,Dropout,Dense
from tensorflow.keras import backend as K

In [ ]:
class SmallerVGGNet:

  @staticmethod
  def build(width, height, depth, num_classes, finalAct = 'softmax'):

    ''' The depth= specifies the number of channels in an input image,
        and num_classes= is the number (integer) of categories/classes (not the class labels themselves).
        We’ll use these parameters in our training script to instantiate the model with a 96 x 96 x 3 input volume. '''

    # initialize the model along with the input shape to be
    # "channels last" and the channels dimension itself
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    # Since we’re using the TensorFlow backend,
    # we arrange the input shape with “channels last” data ordering,
    # but if you want to use “channels first” (Theano, etc.)

    # if we are using "channels first", update the input shape
    # and channels dimension
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
      chanDim = 1

    # CONV => RELU => POOL
    model.add(Conv2D(32, (3,3), padding = 'same', input_shape = inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))


    # softmax classifier
    model.add(Dense(num_classes))
    ''' finalAct dictates whether we’ll use "softmax" activation for single-label classification
        or "sigmoid" activation in the case of today’s multi-label classification. '''
    model.add(Activation(finalAct))

    # return the constructed network architecture
    return model



In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from imutils import paths
import tensorflow as tf
import numpy as np
import random
import pickle
import cv2
import os




# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 30
INIT_LR = 1e-3
BS = 16
IMAGE_DIMS = (96, 96, 3)

# disable eager execution
tf.compat.v1.disable_eager_execution()




################################     loading and preprocessing our training data:      ###################################


imagePaths = sorted(list(paths.list_images('/content/dataset')))
random.seed(42)
random.shuffle(imagePaths)

# initialize the data and labels
data = []
labels = []

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)

    # extract set of class labels from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2].split("_")       # os.path.sep ===>  '/'
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# print("[INFO] data matrix: {} images ({:.2f}MB)".format(
#	len(imagePaths), data.nbytes / (1024 * 1000.0)))

# binarize the labels using scikit-learn's special multi-label
# binarizer implementation
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

# loop over each of the possible class labels and show them
#for (i, label) in enumerate(labels):
#    print(f"{i + 1}. {label}")


# partition the data into training and testing splits using 80% of the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)


# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")



In [ ]:
# initialize the model using a sigmoid activation as the final layer
# in the network so we can perform multi-label classification

model = SmallerVGGNet()
model = model.build(
    width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
    depth=IMAGE_DIMS[2], num_classes=len(labels[1]),
    finalAct="sigmoid")


model.compile(loss="binary_crossentropy", optimizer = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS), metrics=["accuracy"])


# train the network
print("[INFO] training network...")
H = model.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/30
9/9 [==============================] - 9s 669ms/step - batch: 4.0000 - size: 16.0000 - loss: 0.5786 - accuracy: 0.7350 - val_loss: 0.6322 - val_accuracy: 0.6991
Epoch 2/30
9/9 [==============================] - 12s 1s/step - batch: 4.0000 - size: 16.0000 - loss: 0.3661 - accuracy: 0.8715 - val_loss: 0.6702 - val_accuracy: 0.5741
Epoch 3/30
9/9 [==============================] - 12s 1s/step - batch: 4.0000 - size: 16.0000 - loss: 0.2784 - accuracy: 0.9005 - val_loss: 0.7833 - val_accuracy: 0.6250
Epoch 4/30
9/9 [==============================] - 7s 795ms/step - batch: 4.0000 - size: 16.0000 - loss: 0.2402 - accuracy: 0.9074 - val_loss: 0.7447 - val_accuracy: 0.5880
Epoch 5/30
9/9 [==============================] - 6s 626ms/step - batch: 4.0000 - size: 16.0000 - loss: 0.1825 - accuracy: 0.9317 - val_loss: 0.8391 - val_accuracy: 0.7037
Epoch 6/30
9/9 [==============================] - 6s 681ms/step - batch: 4.0000 - size: 16.0000 - loss: 0.1949 - accu

In [ ]:
print("[INFO] serializing network...")

model.save("multi_class.hdf5")

# save the multi-label binarizer to disk
print("[INFO] serializing label binarizer...")

with open("model_labels", "wb") as f:
  f.write(pickle.dumps(mlb))
  f.close()

[INFO] serializing network...
[INFO] serializing label binarizer...


In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("plot")

### Applying Keras multi-label classification to new images

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os



image = cv2.imread("/content/example_03.jpg")

# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)


# load labels
with open("/content/model_labels", 'rb') as f:
  mlb = pickle.load(f)

# load model achitecture and weights
model = load_model("/content/multi_class.hdf5")

predict = model.predict(image)[0]
idxs = np.argsort(predict)[::-1][:2]



/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
